## CAD Tutorial 

In this tutorial you will learn how to create a geometry in [FreeCAD](https://www.freecadweb.org/) and then export and mesh it using [Gmsh](http://gmsh.info/). 

Gmsh provides a build-in CAD engine for defining a geometry, as shown in tutorial BLABLABLA, but using a parametric CAD program such as FreeCAD is much more intuitive and flexible.

Install FreeCAD and Gmsh into your conda environment by running:
conda.yaml

This tutorial is based on an ERT modeling and inversion example on a small dike. However, this FreeCAD → Gmsh workflow can easily be translated to other geophysical methods. The geometry and acquisition design come from the IDEA League [master thesis](https://repository.tudelft.nl/islandora/object/uuid%3A9bf85910-4939-4159-968b-ef558a6ecb7c) of Joost Gevaert. The target in this example is to find the geometry of a sand channel underneath the dike.

### FreeCAD: create the geometry

Two geometries have to be created. One for modeling and one for inversion. When the same meshes are used for modeling and inversion, the geometry of the sand channel is alreadyincluded in the structure of the mesh. Therefore, the mesh itself would act as prior information
to the inversion. The modeling geometry consists of 3 regions: (1) the outer region; (2) the inner
region (same as inversion region in this example) and (3) the sand channel. The inversion geometry consists of 2 regions: (1) the outer region and (2) the inversion region. 

The geometries are defined in three steps:
1. Each region of the geometry designed separately in the Part workbench, or in the Part Design workbench for more complicated geometries. To get familiar with the part design workbench, this [FreeCAD-tutorial](https://wiki.freecadweb.org/Basic_Part_Design_Tutorial_017) with some videos is great. 
2. Merge all regions into one single ”compsolid”, i.e.composite solid. Meaning one object that consists of multiple solids that share the interfaces between the solids.
3. Export the geometry in `.brep` or `.step` format.[<sup>1</sup>](#fn1)


<span id="fn1"> <sup>1</sup></span> `.brep` is preferred, as that is the native format of the OpenCascade CAD engine on which both FreeCAD and Gmsh run. `.step` (also `.stp`) is the standardized CAD exchange format and also works well. Gmsh can also read `.stl` and `.iges` files. `.stl` files only contain surface information and cannot easily be reedited. `.iges` is an old format for which development stopped after 1996 and geometries are not always imported correctly.

(1) The outer and inversion regions of this dike example were created in the Part Design workbench, by making a sketch and then extruding it with the Pad option. See the Inversion-Region in the object tree in the figure below, or the `outer_region.FCStd` and `inversion_region.FCStd` FreeCAD files, ATTACHED!!!. The sand channel is a simple cube, created in the Part workbench. Dimensions: L = 8.0 m ; W = 15.0 m ; H = 2.0 m. Position: x = 7.5 m ; y = -1.5 m ; z = -2.3 m.

(2) The trick then lies in merging these shapes into a single compsolid. This is done in the following steps:

1. Open a new project and merge all objects, i.e. regions (File → Merge project...) into this project
2. In the Part workbench, select all objects and create Boolean Fragments (Part → Split → Boolean Fragments)
3. Select the newly created BooleanFragments in the object tree and change its Mode property to CompSolid, see the figure below.
4. Keep BooleanFragments selected and then apply a Compound Filter to it (Part → Compound → Compound Filter)
5. Quality check the obtained geometry.  
   Select the newly created CompoundFilter from the object tree and click Check Geometry (Part → Check Geometry).  
   SOLID: in the Shape Content, should match the number of objects merged when creating the Boolean Fragments, 3 in this example. 
   COMPSOLID: should be 1. Always, also for other geometries.  
   COMPOUND: should be 0. Always.  
   COMPSOLID: 1 and COMPOUND: 0 indicates that the objects were indeed merged correctly to one single compsolid, see the figure below.

(3) Select the CompounSolid from the object tree and export (File → Export...) as .brep.

![freecad_fig](./freecad_fig.png)

### Gmsh: mesh the geometry

Meshing with Gmsh is incredibly versatile, but has a very steep learning curve. Here we use the Python Application Programming Interface (API). To get familiar with the Python API, the Gmsh [tutorials][tuts] ([overview][t_overview]) were converted to [Python scripts][t_api] and additional [demos][api_demo] are also provided. I will mention or provide links to relevant tutorials and demos, have a look at these for extra context.


[tuts]: https://gitlab.onelab.info/gmsh/gmsh/-/tree/master/tutorial
[t_overview]: http://www.cfdyna.com/Home/gmshCatalogue.html
[t_api]: https://gitlab.onelab.info/gmsh/gmsh/-/tree/master/tutorial/python
[t1]: https://gitlab.onelab.info/gmsh/gmsh/-/blob/master/tutorial/python/t1.py
[api_demo]: https://gitlab.onelab.info/gmsh/gmsh/-/tree/master/demos/api

In [ ]:
from pathlib import Path
import gmsh

# Starting it up (tutorial t1.py)
gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.add("dike_mod")

# Set units to meters
gmsh.option.setString('Geometry.OCCTargetUnit', '')

# Load a BREP or STEP file (t20.py & demo step_assembly.py)
path = Path.cwd().joinpath('dike_mod2.step')
v = gmsh.model.occ.importShapes(str(path))
gmsh.model.occ.synchronize()

# Generate mesh
gmsh.model.mesh.generate(3)

# Run the Gmsh application to verify that importing the geometry went correctly.
gmsh.fltk.run()
gmsh.finalize()